# LangChain: Models, Prompts and Output Parsers

## Introduction

This tutorial from DeepLearning.AI shows us how we can harness the power of OpenAI's GPT-3.5 Turbo model through the utilization of models, prompts, and parsers.

In the next few sections, we will explore how these three essential components work together to create intelligent applications. 

By understanding models, crafting effective prompts, and leveraging LangChain abstractions, we'll gain the skills to build versatile and responsive applications powered by state-of-the-art language processing capabilities.

In [1]:
import os
import openai

In [2]:
with open('keys.txt') as f:
    lines = f.readlines()

openai.api_key = lines[0]

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [3]:
# Create helper function for prompt output
def get_completion(prompt, model='gpt-3.5-turbo'):
    messages = [{'role':'user', 'content':prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0,)
    return response.choices[0].message['content']

In [4]:
get_completion('Who are the founders of OpenAI?')

'The founders of OpenAI are Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman, and Wojciech Zaremba.'

### Example with email response: Complaint

Below we have a customer complaint that is written in 'Pirate' English. We'll convert them into two different versions: American English, and King James English.

In [5]:
# Customer email with complaint in the Pirate English
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

Define the styles we want to apply to our email:

In [6]:
style_1 = """American English \
in a calm and respectful tone
"""

In [7]:
style_2 = """King James English \
in a calm and respectful tone
"""

In [8]:
prompt_1 = f""" Translate the text \
that is delimited by triple backmarks into a style that is {style_1}.
text: ``` {customer_email}```
"""

print(prompt_1)

 Translate the text that is delimited by triple backmarks into a style that is American English in a calm and respectful tone
.
text: ``` 
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [9]:
prompt_2 = f""" Translate the text \
that is delimited by triple backmarks into a style that is {style_2}.
text: ``` {customer_email}```
"""

print(prompt_2)

 Translate the text that is delimited by triple backmarks into a style that is King James English in a calm and respectful tone
.
text: ``` 
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



Outputs of our email complaints:

In [10]:
response_1 = get_completion(prompt_1)
response_1

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!"

In [11]:
response_2 = get_completion(prompt_2)
response_2

'Verily, I am vexed that mine blender lid hath flown off and bespattered mine kitchen walls with smoothie! And to compound mine troubles, the warranty doth not extend to the expense of cleansing mine kitchen. I do beseech thy aid forthwith, good sir!'

## Chat API : LangChain

Let's see how we can do the same using LangChain.

In [12]:
from langchain.chat_models import ChatOpenAI

### Model

We know that we can control the randomness and creativity of the generated text by tweaking the `temperature`. We'll initiate 2 models with identical paramters, other than their temperatures.

In [13]:
# Less random model: Set temperature = 0.0
chat = ChatOpenAI(openai_api_key = openai.api_key, temperature = 0)

In [14]:
# More random model: Set temperature = 0.8
chat_hi_temp = ChatOpenAI(openai_api_key = openai.api_key, temperature = 0.8)

### Prompt Template

Prompts can be long and detailed. By using prompt templates, we can save time when we've constructed an effective one that meets our purposes.

We'll go through an example of how we store a template using `ChatPromptTemplate`.

We start off with are creating a prompt template, `template_string`, that asks the LLM to translate an input `text` into a certain `style` that we want. 

In [15]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

Import `ChatPromptTemplate` to re-use the prompt template above:

In [16]:
from langchain.prompts import ChatPromptTemplate

# Feed the template_string in as the argument
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template

ChatPromptTemplate(input_variables=['text', 'style'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True), additional_kwargs={})])

When we extract the `prompt`, we see the 2 input variables, `style` and `text`, that we included in our prompt earlier.

In [17]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [18]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [19]:
prompt_template.messages[0].prompt.template

'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'

### Example with email response: Customer Complaint

We'll resue the same `customer_style` and `customer_email` from above:

In [20]:
customer_style = """American English \
in a calm and respectful tone
"""

In [21]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [22]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

Notice that the output is a list:

In [23]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


The first element of the list is the prompt, `template_string`, we fed it:

In [24]:
customer_messages[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, example=False)

Let's feed our customer message into the two models we initiated earlier.

In [25]:
customer_response = chat(customer_messages)
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


In [26]:
customer_response_hi_temp = chat_hi_temp(customer_messages)
print(customer_response_hi_temp.content)

I am quite frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add to the difficulties, the warranty does not cover the expenses for cleaning up my kitchen. I would greatly appreciate your assistance at this moment, my friend.


The output we get is a translation to polite American English. It appears that there isn't much difference. Perhaps this is because the email input is short and clear, leaving little room for much variation.

### Example with email response: Customer Service Reply

In [27]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [28]:
service_style = """\
a polite tone \
that speaks in King James English, \
with undertones of pomposity 
"""

In [29]:
service_messages = prompt_template.format_messages(
    style=service_style,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in King James English, with undertones of pomposity 
. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [30]:
service_response = chat(service_messages)
print(service_response.content)

Hearken, goodly customer, verily I say unto thee, the warranty doth not extend its benevolent protection to the expenses incurred in the cleansing of thy kitchen, forsooth! For it is thy very own folly that hath led thee astray, in thy misuse of the blender, by neglecting to place the lid thereupon ere commencing its operation. Alas, thou art left to thy own devices! Fare thee well, and mayhap we shall meet again anon!


In [31]:
service_response_hi_temp = chat_hi_temp(service_messages)
print(service_response_hi_temp.content)

Harken, my goodly patron, disquiet thyself not, forsooth, for it is with a heavy heart that I must impart upon thee this grave tidings. Verily, the warranty, that sacred covenant betwixt us, doth not extend its benevolence unto the expenses of cleaning thy kitchens. Alas, the fault lies not upon the divine essence of thy blender, but upon thine own transgressions, whereby thou didst foolishly neglect to secure the lid ere commencing the blending process. 'Tis a cruel twist of fate, I warrant thee! Farewell and adieu, dear compatriot!


There is a more noticeable difference in the model outputs when we request for a translation in King James English. This is probably due to it's more expressive and colourful nature, which allows the LLM more room to play around with words.

Another possible reason is that LLMs are mostly trained with American English. And as their learning is reinforced by human feedback, they've become fine-tuned to yield a standardized English style.

## Output Parsers

To make our output more readable, we can define the format that we want the LLM's outputs in.

Let's start with defining how we want our information from the LLM to be formatted:

In [32]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

Below we have a sample customer review, `customer_review`, as well as a target template, `review_template`.

In [33]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

Firstly, our desired output format, `review_template`, asks the LLM to take the customer review as input.

Next, it asks the LLM to extract three fields from it and then format the output as JSON with the following keys:
- `gift`
- `delivery_days`
- `price_value`

In [34]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [35]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [36]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, openai_api_key = openai.api_key)
response = chat(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [37]:
type(response.content)

str

Although the response appears like a JSON format with key-value pairs, the output is actually just one long string.

Hence, we'll get an error by running the line of code below because 'gift' is not a dictionary key:

In [38]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

Fortunately, LangChain has an output parser that enables us to format our instructions. 

We start this section by importing `ResponseSchema` and `StructuredOutputParser`.

In [39]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

Langchain provides the `ResponseSchema` class to help return structured output from agents:

In [40]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

Next, we create a `StructuredOutputParser` with the schema:

In [41]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [42]:
print(output_parser)

response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')]


We then get `format_instructions` which helps specify the format of of our instruction and the output of the LLM:

In [43]:
format_instructions = output_parser.get_format_instructions()

In [44]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


Below is our new review template. In our previous template, our instruction for formatting the output was this:
```
Format the output as JSON with the following keys:
gift
delivery_days
price_value
```

This time, in it's place, we have `format_instructions` defined at the bottom:

In [45]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

# Format the prompt
messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

The actual prompt that will be inputted:

In [46]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [47]:
# Call the OpenAI endpoint with `chat`
response = chat(messages)

The output from the LLM is a dictionary in JSON:

In [48]:
print(response.content)

```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


LangChain's `output_parser` allows us to extract the values easily:

In [49]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': False,
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [50]:
type(output_dict)

dict

In [51]:
output_dict.get('delivery_days')

'2'

## Conclusion

This tutorial covered the essential concepts in using LangChain, including models, prompts, and parsers. Models are the language models powering the system, prompts are input instructions for these models, and parsers help structure and extract information from the model's outputs. 

LangChain simplifies the process of creating reusable prompts and output parsers, making it a valuable tool for building applications with large language models. With these tools at our disposal, we can efficiently harness the power of language models for various tasks, from language translation to data extraction, streamlining our development process.